In [56]:
import tensorflow as tf
import numpy as np

# Filt-Filt operation definition

- FeedBack filter components `[M]`:   $A = a_0, a_1, a_2 \dots a_M$, where $a_0 = 0$ by definition
- FeedForward filter components`[N]`:   $B = b_0, b_1, b_2 \dots b_N$

```
y(t) = x(t)*b(0) + x(t-1)*b(1) + x(t-2)*b(2) ... x(t-M)*b(N)
                 - y(t-1)*a(1) - y(t-2)*a(2) ... y(t-N)*a(M)
```

$$
y_t = \sum_{i=0}^{N} x_{t-i} \cdot b_{i} - \sum_{i=1}^{M} y_{t-i} \cdot a_{i}
$$


```
y(t) = x[t-N : t] * b[::-1] - y[t-M : t-1] * a[::-1]
```

for faster computation let $s$ be $s := x \ast b$, where $\ast$ is the convolution operator, since all values of $x$ is known at computation time

```
y(t) = h[t] - y[y-t : t-1] * a[::-1]
```


In [57]:
def resetSession():
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    return sess

In [58]:
sess = resetSession()
# Input
x = tf.placeholder(tf.float32, shape=[None], name='x_vector')

# Feedback filter
a = tf.placeholder(tf.float32, shape=[None], name='a_vector')
# Feedforward filter
b = tf.placeholder(tf.float32, shape=[None], name='b_vector')

# Support vector
# Default convolution works like this
# [... 1  2  3  4  5...]
#      *  *  *  *  *
# [    a  b  c  d  e    ]
#            +
#         result

# But this time we need the rightmost value as a result
# [... 1  2  3  4  5...]
#      *  *  *  *  *
# [    a  b  c  d  e    ]
#                  +
#               result
#
# So we use padding like a Jedi, and use VALID convolution
x_padded = tf.pad(x, [[tf.shape(b)[0]-1, 0]])

# value[None, :, None] stands for [Batch, Sample, Channels]
# In the basic example Batch = Channel = 1
# filters[::-1, None, None] stands for [Filter_size(inverted), In_size, Out_size]
# In the basic example In_size = Out_size = 1
s = tf.nn.conv1d(value=x_padded[None, :, None], filters=b[::-1, None, None], stride=1, padding='VALID')
s = tf.squeeze(s, name='s_vector')

In [59]:
x_test = list(range(1, 5))
b_test = [0, 1, 0]

x_padded.eval({b:b_test, x:x_test})

array([ 0.,  0.,  1.,  2.,  3.,  4.], dtype=float32)

In [60]:
s.eval({x:x_test, b:b_test})

array([ 0.,  1.,  2.,  3.], dtype=float32)

In [61]:
# Watch out, the filter is now inverted
b_test = [0, 0, 1]
s.eval({x:x_test, b:b_test})

# [... 1  2  3  4  5...]
#      *  *  *  *  *
# [    e  d  c  b  a   ]
#                  +
#               result

array([ 0.,  0.,  1.,  2.], dtype=float32)

In [62]:
b_test = [1, 0, 0]
s.eval({x:x_test, b:b_test})

array([ 1.,  2.,  3.,  4.], dtype=float32)

In [63]:
# Output
# Y values can be evaluated one-by-one, so as soon as we evaluate the t^th value
# We push it to the back of the queue, and we always take as many values
# as long is the Feedback filter vector is, for efficiently calculating inner product

# Capacity is the upper bound of the size of the filter, 
# Since the queue needs to be able to dequeue filter sized vectors at once.

filter_size_upper_bound = 100
CAPACITY = 3 * filter_size_upper_bound
# Since we could enqueue multiple tensors we use 
# listed `shapes` and `dtypes` but that is only for syntax autofellatio
# Basically we will enqueue and dequeue scalar values (shape=[], dtype=tf.float32)
y_queue = tf.FIFOQueue(CAPACITY, dtypes=[tf.float32], shapes=[[]], name='y_fifo')

# TF QUEUE mechanism demonstration

**Supplementary reading [TF guide on queues](https://www.tensorflow.org/programmers_guide/threading_and_queues)**

![demo](https://www.tensorflow.org/images/IncremeterFifoQueue.gif)

In [64]:
# We initialize the queue with zeros to mimic padding
init = y_queue.enqueue_many((tf.zeros(tf.shape(b))))

# y_deq is technically y[t-M : t-1] 
# which will be used for the inner product operation
y_deq = y_queue.dequeue_many(tf.shape(b))

In [65]:
print(sess.run([init, y_deq], {b:b_test}))
# The result is
# [None, array([ 0.,  0.,  0.], dtype=float32)]
# because evaluating the init operator does not return any value
# and evaluating the y_prev just returns our initialized values
# be careful! now the queue is empty again

# TRY ME!
# print(sess.run([y_queue.enqueue((1)), init, y_prev], {b:b_test}))
# now the queue is not empty, one zero is still left in it
# be careful! it remembers its state within a Session
# print(sess.run([y_queue.enqueue((1)), init, y_prev], {b:b_test}))

[None, array([ 0.,  0.,  0.], dtype=float32)]


In [66]:
# a single vector product... a bit too verbose
print(sess.run(tf.reduce_sum(a * b), {a:[1, 2, 3], b:[1, 10, 100]}))
# reduce it
def prod(a, b, **kwargs):
    return tf.reduce_sum(a * b, **kwargs)

321.0


In [67]:
# evaluating y_1, using s_t with zero indexing is s[t-1]
# Now we write by hand the first step, 
# but this is not necessary
y_1 = s[0] - prod(y_deq, a) 

In [68]:
x_test = list(range(1, 5))
b_test = [1, 0, 0]
a_test = [1, 0, 0]
test_feed = {x:x_test, b:b_test, a:a_test}

sess.run([init, s, y_deq, y_1], test_feed)

[None,
 array([ 1.,  2.,  3.,  4.], dtype=float32),
 array([ 0.,  0.,  0.], dtype=float32),
 1.0]

In [69]:
# now evaluate a few more y_t
# and see that to do it by hand is very annoying
y_1 = s[0] - prod(y_deq, a) 
enq1 = y_queue.enqueue(y_1)

y_2 = s[1] - prod(y_deq, a)
enq2 = y_queue.enqueue(y_2)

y_3 = s[2] - prod(y_deq, a)
enq3 = y_queue.enqueue(y_3)

y_4 = s[3] - prod(y_deq, a)

explicit_fetch_list = [
    init,
    y_1,
    enq1,
    y_2,
    enq2,
    y_3,
    enq3,
    y_4,
]
sess.run(explicit_fetch_list, test_feed)

[None, 1.0, None, 2.0, None, 3.0, None, 4.0]

In [72]:
# IMPORTANT NOTE!!!
# The order of graph definition here is not relevant
# These operations are not executed by calling the lines above
# the evaluation order is determined in the _fetch_list
# 
# fetch is the operation when you evaluate a tensor
# fetch its value (in the form of a numpy array) 
# from the computational graph
#
# The python variables here can be represented as
# C++ pointers... so here each operator is pointed by y_i
#
# If I wrote simply y instead
# like this

y = s[0] - prod(y_deq, a) 
enq1 = y_queue.enqueue(y)

y = s[1] - prod(y_deq, a)
enq2 = y_queue.enqueue(y)

y = s[2] - prod(y_deq, a)
enq3 = y_queue.enqueue(y)

y = s[3] - prod(y_deq, a)

impl_fetch_list = [
    y_deq,
    init,
    enq1,
    enq2,
    enq3,
    y
]

# In this case y always points to the latest operation/tensor we defined
# It is still working because enq1, enq2... are linked together with
# different y_t tensors

# In order to make the computation valid
# We need to enqueue in the right order

# The first operator ensures that the list is empty
sess.run(impl_fetch_list, test_feed)


[array([ 0.,  0.,  0.], dtype=float32), None, None, None, None, 4.0]